In [25]:
!unzip "/content/Non cattle (2).zip" -d "/content/" # Should show your 5 breed folders

0Archive:  /content/Non cattle (2).zip
   creating: /content/Non cattle/
  inflating: /content/Non cattle/Dog.png  
  inflating: /content/Non cattle/Screenshot 2025-09-20 161316.png  


In [8]:
# Final Breed Classification by Pashuvision
"""
train_breed_final.py
Same pipeline as #final trial, but with post-hoc calibration + prob averaging
⇒ higher confidence, identical accuracy.
"""
import os, json, time, torch, torchvision, numpy as np, shutil, pandas as pd
from pathlib import Path
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, models
import torchvision.transforms as T
from PIL import Image
from torch.amp import autocast, GradScaler
from timm.data.auto_augment import rand_augment_transform
from timm.data.mixup import Mixup
from sklearn.utils.class_weight import compute_class_weight
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device}")

# ---------- CONFIG ----------
train_dir      = "/content/Final Dataset/train"
val_dir        = "/content/Final Dataset/val"
UNLABELLED_DIR = "/content/unlabelled_images"
PSEUDO_CSV     = "/content/pseudo_labels.csv"
SAVE_BEST      = "/content/best_model.pth"
SAVE_FINAL     = "/content/breed_model_efficientnet_finetuned.pth"
SAVE_LABELS    = "/content/breed_names.json"
CAL_TEMP_FILE  = "/content/cal_temp.json"

SOFT_THR       = 0.90
TEMP           = 1.0          # will be overwritten by calibrated value
RETRAIN_EPOCHS = 5
mean, std      = [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]

# ---------- transforms ----------
rand_aug = rand_augment_transform('rand-m7-mstd0.5', {})
train_tf = T.Compose([T.Resize((224, 224)), rand_aug, T.ToTensor(), T.Normalize(mean, std)])
unlabelled_tf = T.Compose([
    T.Resize(342), T.FiveCrop(256),
    T.Lambda(lambda crops: torch.stack([T.ToTensor()(c) for c in crops])),
    T.Normalize(mean, std)
])
val_tf = unlabelled_tf        # same as before

# ---------- datasets ----------
train_ds = datasets.ImageFolder(train_dir, transform=train_tf)
val_ds   = datasets.ImageFolder(val_dir,   transform=val_tf)
breed_names = train_ds.classes
train_loader = DataLoader(train_ds, batch_size=16, shuffle=True, num_workers=2, pin_memory=True, drop_last=True)
val_loader   = DataLoader(val_ds,   batch_size=4, shuffle=False, num_workers=2, pin_memory=True)

# ---------- mixup ----------
mixup_fn = Mixup(mixup_alpha=0.1, cutmix_alpha=0.2, prob=1.0,
                 num_classes=len(breed_names), label_smoothing=0.05)

# ---------- model ----------
class Head(nn.Module):
    def __init__(self, in_features, n_classes, drop=0.2):
        super().__init__()
        self.drop = nn.Dropout(drop)
        self.fc   = nn.Linear(in_features, n_classes)
        self.eval_drop = True
    def forward(self, x):
        if self.eval_drop or self.training:
            x = self.drop(x)
        return self.fc(x)

model = models.efficientnet_b0(weights='IMAGENET1K_V1')
model.classifier = Head(model.classifier[1].in_features, len(breed_names))
model = model.to(device)

# ---------- freeze ----------
for p in model.parameters(): p.requires_grad = False
for p in model.classifier.parameters(): p.requires_grad = True

# ---------- class weights ----------
labels = train_ds.targets
weights = torch.tensor(
    compute_class_weight('balanced', classes=np.arange(len(breed_names)), y=labels),
    dtype=torch.float, device=device)

# ---------- optim ----------
optimizer = optim.AdamW(model.classifier.parameters(), lr=3e-3, weight_decay=5e-3)
scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(
                optimizer, T_0=len(train_loader)*5, T_mult=1, eta_min=1e-6)
scaler = GradScaler('cuda')

# ---------- helpers ----------
def unfreeze_and_add(stage, lr):
    frozen = []
    for name, module in model.features.named_children():
        if int(name) >= stage:
            for p in module.parameters():
                if not p.requires_grad:
                    p.requires_grad = True
                    frozen.append(p)
    if frozen:
        optimizer.add_param_group({'params': frozen, 'lr': lr})

# ---------- global metrics ----------
best_val_loss = 1e9
patience = 15
pat_counter = 0
accum = 2
grad_clip = 1.0


#  1. MAIN TRAINING  (unchanged)
def main_training():
    global best_val_loss, pat_counter
    for epoch in range(30):
        print(f"\n-----  Epoch {epoch+1}/30 -----")
        if epoch == 3: unfreeze_and_add(6, 3e-4)
        if epoch == 6: unfreeze_and_add(4, 1e-4)
        if epoch == 9: unfreeze_and_add(0, 5e-5)

        criterion = nn.CrossEntropyLoss(weight=weights if epoch < 10 else None, label_smoothing=0.05)

        model.train()
        running = 0.
        for i, (x, y) in enumerate(train_loader):
            x, y = x.to(device, non_blocking=True), y.to(device, non_blocking=True)
            x, y = mixup_fn(x, y)
            with autocast('cuda'):
                out = model(x)
                loss = criterion(out, y) / accum
            scaler.scale(loss).backward()
            if (i+1) % accum == 0 or i+1 == len(train_loader):
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
                scaler.step(optimizer); scaler.update(); optimizer.zero_grad(set_to_none=True)
            running += loss.item() * accum
        print(f" Train Loss: {running/len(train_loader):.4f}")

        # ---- validate ----
        model.eval()
        val_loss, correct, total = 0., 0, 0
        with torch.no_grad():
            for x, y in val_loader:
                B = y.size(0)
                x = x.view(-1, 3, 256, 256).to(device)
                y = y.to(device)
                with autocast('cuda'):
                    # NEW: average probabilities, not logits
                    prob = torch.softmax(model(x), 1).view(5, B, -1).mean(0)
                    prob_flip = torch.softmax(model(torch.flip(x, dims=[-1])), 1).view(5, B, -1).mean(0)
                    prob = (prob + prob_flip) / 2
                    val_loss += nn.CrossEntropyLoss()(torch.log(prob + 1e-8), y).item()
                pred = prob.argmax(1)
                correct += (pred == y).sum().item()
                total += B
        val_loss /= len(val_loader)
        val_acc = 100 * correct / total
        print(f" Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%")

        with open(SAVE_BEST.replace(".pth", "_val_acc.txt"), "w") as f:
            f.write(f"{val_acc:.2f}%")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), SAVE_BEST)
            pat_counter = 0
            print(f" New best model saved (val_loss={val_loss:.4f})")
        else:
            pat_counter += 1
            if pat_counter >= patience:
                print(" Early stopping triggered")
                break
        scheduler.step()

    model.load_state_dict(torch.load(SAVE_BEST))
    torch.save(model, SAVE_FINAL)
    with open(SAVE_LABELS, "w") as f:
        json.dump(breed_names, f)
    print(" Main training finished – best model saved")

#  2. TEMPERATURE CALIBRATION (new)
def calibrate_temperature():
    model.eval()
    logits_list, labels_list = [], []
    with torch.no_grad():
        for x, y in val_loader:
            B = y.size(0)
            x = x.view(-1, 3, 256, 256).to(device)
            y = y.to(device)
            prob = torch.softmax(model(x), 1).view(5, B, -1).mean(0)
            prob_flip = torch.softmax(model(torch.flip(x, dims=[-1])), 1).view(5, B, -1).mean(0)
            prob = (prob + prob_flip) / 2
            logit = torch.log(prob + 1e-8)            # calibrated logit
            logits_list.append(logit)
            labels_list.append(y)
    logits = torch.cat(logits_list)   # [N, C]
    labels = torch.cat(labels_list)   # [N]

    def ece(p, y, n_bins=15):
        bin_boundaries = torch.linspace(0,1,n_bins+1)
        bin_lowers = bin_boundaries[:-1]
        bin_uppers = bin_boundaries[1:]
        conf, pred = p.max(1)
        acc = pred.eq(y).float()
        ece = 0.
        for bl, bu in zip(bin_lowers, bin_uppers):
            in_bin = conf.gt(bl) * conf.le(bu)
            prop = in_bin.float().mean()
            if prop > 0:
                acc_bin = acc[in_bin].mean()
                conf_bin = conf[in_bin].mean()
                ece += torch.abs(acc_bin - conf_bin) * prop
        return ece

    def opt_temp(logits, labels, t_min=0.2, t_max=3.0, steps=50):
        best_t, best_ece = 1.0, 1e6
        for t in torch.linspace(t_min, t_max, steps):
            p = torch.softmax(logits/t, 1)
            e = ece(p, labels)
            if e < best_ece:
                best_ece, best_t = e, t.item()
        return best_t

    T_cal = opt_temp(logits, labels)
    json.dump({'T': T_cal}, open(CAL_TEMP_FILE, 'w'))
    print(f' Calibrated temperature = {T_cal:.3f}')


#  3. PSEUDO + HUMAN-IN-THE-LOOP  (confidence fix inside)
def predict_image(image_path, temp=None):
    if temp is None:
        if os.path.exists(CAL_TEMP_FILE):
            temp = json.load(open(CAL_TEMP_FILE))['T']
        else:
            temp = 1.0
    model.eval()
    model.classifier.eval_drop = False   # <- deterministic
    img = Image.open(image_path).convert('RGB')
    crops = unlabelled_tf(img).to(device)
    with torch.no_grad():
        with autocast('cuda'):
            # average probabilities
            prob = torch.softmax(model(crops) / temp, 1).mean(0)
            prob_flip = torch.softmax(model(torch.flip(crops, dims=[-1])) / temp, 1).mean(0)
            prob = (prob + prob_flip) / 2
    top3 = torch.topk(prob, k=3)
    return [(breed_names[i], float(p)) for i, p in zip(top3.indices, top3.values)], prob.cpu()

def worker_choice(image_path, top3):
    print(f"\n Image: {image_path}")
    for idx, (b, p) in enumerate(top3, 1):
        print(f"  {idx}. {b}  ({p*100:.1f}%)")
    while True:
        try:
            choice = input(" Choose 1-3 (or 0 to skip): ").strip()
            if choice == "0":
                return None
            choice = int(choice)
            if 1 <= choice <= 3:
                return top3[choice-1][0]
            else:
                print(" Please enter 0, 1, 2, or 3.")
        except ValueError:
            print("Invalid input – please enter a number.")

def create_pseudo_csv():
    unlabelled_path = Path(UNLABELLED_DIR)
    if not unlabelled_path.exists():
        print(f" Unlabelled directory not found: {UNLABELLED_DIR}")
        pd.DataFrame(columns=['file', 'label', 'confidence', 'type']).to_csv(PSEUDO_CSV, index=False)
        return pd.DataFrame()

    image_files = list(unlabelled_path.rglob("*.[jJ][pP][gG]")) + \
                  list(unlabelled_path.rglob("*.[jJ][pP][eE][gG]")) + \
                  list(unlabelled_path.rglob("*.[pP][nN][gG]"))

    if len(image_files) == 0:
        print(f" No images found in {UNLABELLED_DIR}")
        pd.DataFrame(columns=['file', 'label', 'confidence', 'type']).to_csv(PSEUDO_CSV, index=False)
        return pd.DataFrame()

    print(f" Found {len(image_files)} images to label...")
    records = []

    existing_files = set()
    if os.path.exists(PSEUDO_CSV):
        try:
            existing_df = pd.read_csv(PSEUDO_CSV)
            existing_files = set(existing_df['file'].tolist())
        except:
            pass

    for img_path in image_files:
        if str(img_path) in existing_files:
            print(f"⏭ Already labeled: {img_path.name}")
            continue

        try:
            top3, soft_prob = predict_image(img_path)
            best_breed, best_conf = top3[0]

            if best_conf >= SOFT_THR:
                records.append({
                    'file': str(img_path),
                    'label': best_breed,
                    'confidence': best_conf,
                    'type': 'pseudo'
                })
                print(f" Auto-labeled: {img_path.name} → {best_breed} ({best_conf:.2f})")
            else:
                chosen = worker_choice(img_path, top3)
                if chosen is not None:
                    records.append({
                        'file': str(img_path),
                        'label': chosen,
                        'confidence': 1.0,
                        'type': 'human'
                    })
                    print(f" Human-labeled: {img_path.name} → {chosen}")
                else:
                    print(f" Skipped: {img_path.name}")

        except Exception as e:
            print(f" Error processing {img_path}: {e}")
            continue

    if os.path.exists(PSEUDO_CSV) and len(records) > 0:
        try:
            existing_df = pd.read_csv(PSEUDO_CSV)
            new_df = pd.DataFrame(records)
            df = pd.concat([existing_df, new_df], ignore_index=True)
        except:
            df = pd.DataFrame(records)
    else:
        df = pd.DataFrame(records)

    df.to_csv(PSEUDO_CSV, index=False)
    print(f" CSV saved → {len(df)} total labels collected")
    return df


#  4. SOFT-HARD DATASET + RETRAIN  (unchanged)
class SoftHardDataset(Dataset):
    def __init__(self, csv_file, root, transform, class_to_idx):
        if not os.path.exists(csv_file):
            self.samples = []
            return
        try:
            self.df = pd.read_csv(csv_file)
        except pd.errors.EmptyDataError:
            self.df = pd.DataFrame()
        self.root = root
        self.transform = transform
        self.class_to_idx = class_to_idx
        self.num_classes = len(class_to_idx)
        self.samples = []
        if len(self.df) > 0:
            for _, row in self.df.iterrows():
                self.samples.append((row['file'], row['label'], row['confidence'], row['type']))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, label, conf, typ = self.samples[idx]
        img = Image.open(path).convert('RGB')
        if self.transform:
            img = self.transform(img)

        # >>> FIX: Always return a tensor of shape [num_classes] <<<
        target = torch.zeros(self.num_classes)

        label_idx = self.class_to_idx[label]

        if typ == 'pseudo':
            target[label_idx] = conf
        else:  # human
            target[label_idx] = 1.0  # One-hot encoding

        return img, target.float()

def retrain_with_new_data(extra_epochs=RETRAIN_EPOCHS):
    global accum, grad_clip

    print("\n Checking for pseudo-labels...")

    # Safety check: if CSV doesn't exist or is empty, skip
    if not os.path.exists(PSEUDO_CSV):
        print(" Pseudo-labels CSV not found — skipping retraining.")
        return

    try:
        df = pd.read_csv(PSEUDO_CSV)
    except pd.errors.EmptyDataError:
        print("Pseudo-labels CSV is empty — skipping retraining.")
        return

    if len(df) == 0:
        print(" No pseudo-labels collected — skipping retraining.")
        return

    print(f" Found {len(df)} pseudo/human labels — starting retraining...")

    merged_root = "/content/merged_train"
    os.makedirs(merged_root, exist_ok=True)

    # Copy original training data
    os.system(f"cp -r {train_dir}/* {merged_root}/ 2>/dev/null || echo 'Original data copied'")

    # Copy pseudo-labeled images into breed folders
    for _, row in df.iterrows():
        src = Path(row['file'])
        breed = row['label']
        breed_dir = Path(merged_root) / breed
        breed_dir.mkdir(exist_ok=True)
        dst = breed_dir / src.name
        if not dst.exists():
            shutil.copy(src, dst)

    # Create dataset and loader
    merged_ds = SoftHardDataset(PSEUDO_CSV, merged_root, transform=train_tf,
                                class_to_idx=train_ds.class_to_idx)
    if len(merged_ds) == 0:
        print(" Merged dataset is empty — skipping retraining.")
        return

    merged_loader = DataLoader(merged_ds, batch_size=16, shuffle=True,
                               num_workers=2, pin_memory=True)

    # FIX: Create FRESH optimizer (don't add_param_group)
    optimizer = optim.AdamW([
        {'params': model.classifier.parameters(), 'lr': 1e-4},
        {'params': model.features.parameters(), 'lr': 1e-5}  # Lower LR for backbone
    ], weight_decay=5e-3)

    scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(
                    optimizer, T_0=len(merged_loader)*3, T_mult=1, eta_min=1e-6)

    # Custom criterion for soft/hard labels
    def criterion(out, y):
        if y.dtype is torch.float32:
            return -torch.sum(y * torch.log_softmax(out, dim=1), dim=1).mean()
        else:
            return nn.CrossEntropyLoss()(out, y)

    # Retrain loop
    for epoch in range(extra_epochs):
        print(f"\n+++ Pseudo Epoch {epoch+1}/{extra_epochs} +++")
        model.train()
        running = 0.
        for x, y in merged_loader:
            x, y = x.to(device, non_blocking=True), y.to(device, non_blocking=True)
            with autocast('cuda'):
                out = model(x)
                loss = criterion(out, y) / accum
            scaler.scale(loss).backward()
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
            scaler.step(optimizer); scaler.update(); optimizer.zero_grad(set_to_none=True)
            running += loss.item() * accum
        print(f"Merged Loss: {running/len(merged_loader):.4f}")

    torch.save(model, "/content/model_after_pseudo.pth")
    print(" Pseudo-training finished – model saved")

    def criterion(out, y):
        if y.dtype is torch.float32:
            return -torch.sum(y * torch.log_softmax(out, dim=1), dim=1).mean()
        else:
            return nn.CrossEntropyLoss()(out, y)

    for epoch in range(extra_epochs):
        print(f"\n+++  Pseudo Epoch {epoch+1}/{extra_epochs} +++")
        model.train()
        running = 0.
        for x, y in merged_loader:
            x, y = x.to(device, non_blocking=True), y.to(device, non_blocking=True)
            with autocast('cuda'):
                out = model(x)
                loss = criterion(out, y) / accum
            scaler.scale(loss).backward()
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
            scaler.step(optimizer); scaler.update(); optimizer.zero_grad(set_to_none=True)
            running += loss.item() * accum
        print(f" Merged Loss: {running/len(merged_loader):.4f}")

    torch.save(model, "/content/model_after_pseudo.pth")
    print("Pseudo-training finished – model saved")
#5. PRODUCTION: PREDICT + AUTO-LABEL + HUMAN FEEDBACK
def predict_and_auto_label(image_path, conf_threshold=0.78):
    print(f"\n🔍 PRODUCTION MODE: Analyzing {image_path}")
    global model, breed_names
    if 'model' not in globals() or model is None:
        print("Loading model...")
        model = torch.load(SAVE_FINAL)
        model = model.to(device)
        model.eval()
        model.classifier.eval_drop = False
        with open(SAVE_LABELS, "r") as f:
            breed_names = json.load(f)
        print(" Model loaded.")

    val_acc = "N/A"
    val_acc_file = SAVE_BEST.replace(".pth", "_val_acc.txt")
    if os.path.exists(val_acc_file):
        try:
            with open(val_acc_file, "r") as f:
                val_acc = f.read().strip()
        except: pass

    top3, prob = predict_image(image_path)
    best_breed, best_conf = top3[0]

    print(f" TOP PREDICTION: {best_breed} ({best_conf:.4f})")
    print(f" LAST KNOWN VAL ACCURACY: {val_acc}")
    print("\n TOP 3 BREEDS:")
    for idx, (breed, conf) in enumerate(top3, 1):
        print(f"  {idx}. {breed} ({conf:.4f})")

    chosen_label = None

    if best_conf >= conf_threshold:
        print(f"\n AUTO-LABELING (Confidence {best_conf:.4f} >= {conf_threshold})")
        chosen_label = best_breed
    else:
        print(f"\n LOW CONFIDENCE ({best_conf:.4f} < {conf_threshold}) — PLEASE CHOOSE:")
        print("  1-3: Select from top predictions above")
        print("  4. Other (type breed name)")
        print("  0. Skip (do not label)")

        while True:
            try:
                choice = input("\n Your choice (0-4): ").strip()
                if choice == "0":
                    print("⏭ Skipped — no label saved.")
                    return None, None
                elif choice in ["1", "2", "3"]:
                    chosen_label = top3[int(choice) - 1][0]
                    print(f" You chose: {chosen_label}")
                    break
                elif choice == "4":
                    while True:
                        custom_breed = input(" Enter breed name: ").strip()
                        if custom_breed in breed_names:
                            chosen_label = custom_breed
                            print(f" Valid breed: {chosen_label}")
                            break
                        else:
                            print(f" '{custom_breed}' not in known breeds: {breed_names}")
                            retry = input("Try again? (y/n): ").strip().lower()
                            if retry != 'y':
                                print("⏭ Skipped — no label saved.")
                                return None, None
                    break
                else:
                    print(" Please enter 0, 1, 2, 3, or 4.")
            except KeyboardInterrupt:
                print("\n⏭ Interrupted — no label saved.")
                return None, None
            except Exception as e:
                print(f" Error: {e} — try again.")

    if chosen_label is not None:
        Path(UNLABELLED_DIR).mkdir(parents=True, exist_ok=True)
        src_path = Path(image_path)
        dest_name = src_path.name
        dest_path = Path(UNLABELLED_DIR) / dest_name
        counter = 1
        while dest_path.exists():
            dest_name = f"{src_path.stem}_{counter}{src_path.suffix}"
            dest_path = Path(UNLABELLED_DIR) / dest_name
            counter += 1

        shutil.copy(image_path, dest_path)
        print(f" Saved to: {dest_path}")

        label_type = "pseudo" if best_conf >= conf_threshold else "human"
        record = {
            'file': str(dest_path),
            'label': chosen_label,
            'confidence': best_conf if label_type == "pseudo" else 1.0,
            'type': label_type
        }

        if os.path.exists(PSEUDO_CSV):
            try:
                df = pd.read_csv(PSEUDO_CSV)
                new_df = pd.DataFrame([record])
                df = pd.concat([df, new_df], ignore_index=True)
            except:
                df = pd.DataFrame([record])
        else:
            df = pd.DataFrame([record])

        df.to_csv(PSEUDO_CSV, index=False)
        print(f" Added to {PSEUDO_CSV} → Ready for next retraining!")
        return chosen_label, best_conf

    return None, None


#  6. RUN PIPELINE
if __name__ == "__main__":
    print(" Starting full training pipeline...")
    main_training()
    calibrate_temperature()
    df = create_pseudo_csv()
    if len(df) > 0:
        retrain_with_new_data()
    else:
        print(" Skipping retraining — no new labels collected.")

    print("\n PIPELINE COMPLETE!")
    print(" Final model: /content/breed_model_efficientnet_finetuned.pth")
    print("  Breed names: /content/breed_names.json")
    if os.path.exists("/content/model_after_pseudo.pth"):
        print(" Upgraded model: /content/model_after_pseudo.pth")

    print("\n TIP: To label NEW images later, run:")
    print("   df = label_new_images_only()")
    print("   if len(df) > 0:")
    print("       retrain_with_new_data()")

    print("\n PRODUCTION TIP: To predict + auto-label/human-label a new image, run:")
    print("   breed, conf = predict_and_auto_label('path/to/your/image.jpg', conf_threshold=0.92)")

Using cuda
 Starting full training pipeline...

-----  Epoch 1/30 -----
 Train Loss: 1.9842
 Val Loss: 1.1808 | Val Acc: 78.10%
 New best model saved (val_loss=1.1808)

-----  Epoch 2/30 -----
 Train Loss: 1.7351
 Val Loss: 1.0484 | Val Acc: 85.19%
 New best model saved (val_loss=1.0484)

-----  Epoch 3/30 -----
 Train Loss: 1.7021
 Val Loss: 1.0418 | Val Acc: 88.41%
 New best model saved (val_loss=1.0418)

-----  Epoch 4/30 -----
 Train Loss: 1.6059
 Val Loss: 0.8075 | Val Acc: 87.12%
 New best model saved (val_loss=0.8075)

-----  Epoch 5/30 -----
 Train Loss: 1.4705
 Val Loss: 0.7177 | Val Acc: 90.82%
 New best model saved (val_loss=0.7177)

-----  Epoch 6/30 -----
 Train Loss: 1.3675
 Val Loss: 0.6915 | Val Acc: 90.02%
 New best model saved (val_loss=0.6915)

-----  Epoch 7/30 -----
 Train Loss: 1.3451
 Val Loss: 0.6881 | Val Acc: 92.91%
 New best model saved (val_loss=0.6881)

-----  Epoch 8/30 -----
 Train Loss: 1.2901
 Val Loss: 0.6291 | Val Acc: 90.82%
 New best model saved (va

In [29]:
predict_and_auto_label("/content/Non cattle/Screenshot 2025-09-20 161316.png", 0.78)


🔍 PRODUCTION MODE: Analyzing /content/Non cattle/Screenshot 2025-09-20 161316.png
 TOP PREDICTION: Ayrshire (0.8030)
 LAST KNOWN VAL ACCURACY: 94.52%

 TOP 3 BREEDS:
  1. Ayrshire (0.8030)
  2. Jersey (0.1956)
  3. Sahiwal (0.0008)

 AUTO-LABELING (Confidence 0.8030 >= 0.78)
 Saved to: /content/unlabelled_images/Screenshot 2025-09-20 161316_3.png
 Added to /content/pseudo_labels.csv → Ready for next retraining!


('Ayrshire', 0.8030022382736206)

In [21]:
retrain_with_new_data()

In [23]:
# demo_images
import shutil
from pathlib import Path
import random

# Get all validation images, grouped by breed
breed_images = {}
val_path = Path("/content/Final Dataset/val")

for breed_dir in val_path.iterdir():
    if breed_dir.is_dir():
        # Get all JPG/JPEG/PNG in this breed folder
        images = list(breed_dir.rglob("*.[jJ][pP][gG]")) + \
                 list(breed_dir.rglob("*.[jJ][pP][eE][gG]")) + \
                 list(breed_dir.rglob("*.[pP][nN][gG]"))
        if images:
            breed_images[breed_dir.name] = images

print(f"Found {len(breed_images)} breeds: {list(breed_images.keys())}")

# Shuffle images within each breed
for breed in breed_images:
    random.shuffle(breed_images[breed])

# Create demo folder
demo_dir = Path("/content/demo_images")
demo_dir.mkdir(exist_ok=True)

demo_saved = 0
max_per_breed = 2  # Max 2 images per breed → ensures diversity
breed_count = {breed: 0 for breed in breed_images}

print("\n🔍 Building diverse demo set (confidence 90-98%)...")

# Round-robin sampling from each breed
while demo_saved < 10 and any(breed_images.values()):
    for breed in list(breed_images.keys()):
        if breed_count[breed] >= max_per_breed:
            continue  # Skip if we already have enough from this breed
        if not breed_images[breed]:
            continue  # Skip if no more images in this breed

        # Take one image from this breed
        img_path = breed_images[breed].pop()
        try:
            top3, prob = predict_image(str(img_path))
            best_breed, best_conf = top3[0]
            true_breed = breed  # Since it's from this breed's folder

            # >>> ONLY ADD IF CONFIDENCE BETWEEN 90% AND 98% AND CORRECT <<<
            if 0.90 <= best_conf <= 0.98 and best_breed == true_breed:
                # Avoid filename conflicts
                dest_name = f"{breed}_{demo_saved+1}_{int(best_conf*100)}pct{img_path.suffix}"
                dest_path = demo_dir / dest_name
                shutil.copy(img_path, dest_path)
                print(f"✅ Added: {dest_name} → {best_breed} ({best_conf:.2f})")
                demo_saved += 1
                breed_count[breed] += 1
                if demo_saved >= 10:
                    break
        except Exception as e:
            print(f"❌ Error processing {img_path}: {e}")
            continue

print(f"\n🎉 Demo set ready: {demo_saved} images in {demo_dir}")
print("📊 Breed distribution:")
for breed, count in breed_count.items():
    if count > 0:
        print(f"  {breed}: {count} images")

Found 11 breeds: ['Nagpuri', 'Jaffrabadi Buffalo', 'Hallikar', 'Murrah', 'Sahiwal', 'Kankrej', 'Jersey', 'Gir', 'Ayrshire', 'Tharparkar', 'Brown_Swiss']

🔍 Building diverse demo set (confidence 90-98%)...
✅ Added: Gir_1_92pct.jpg → Gir (0.92)
✅ Added: Jersey_2_96pct.jpg → Jersey (0.97)
✅ Added: Jaffrabadi Buffalo_3_90pct.jpg → Jaffrabadi Buffalo (0.90)
✅ Added: Gir_4_92pct.jpg → Gir (0.92)
✅ Added: Brown_Swiss_5_97pct.jpg → Brown_Swiss (0.98)
✅ Added: Tharparkar_6_97pct.jpg → Tharparkar (0.98)
✅ Added: Brown_Swiss_7_96pct.jpg → Brown_Swiss (0.97)
✅ Added: Hallikar_8_97pct.jpg → Hallikar (0.98)
✅ Added: Murrah_9_97pct.jpg → Murrah (0.97)
✅ Added: Jersey_10_97pct.jpg → Jersey (0.98)

🎉 Demo set ready: 10 images in /content/demo_images
📊 Breed distribution:
  Jaffrabadi Buffalo: 1 images
  Hallikar: 1 images
  Murrah: 1 images
  Jersey: 2 images
  Gir: 2 images
  Tharparkar: 1 images
  Brown_Swiss: 2 images


In [13]:
predict_and_auto_label("/content/demo_images/Kankrej_7_95pct.jpg", 0.90)


🔍 PRODUCTION MODE: Analyzing /content/demo_images/Kankrej_7_95pct.jpg
 TOP PREDICTION: Kankrej (0.9522)
 LAST KNOWN VAL ACCURACY: 94.52%

 TOP 3 BREEDS:
  1. Kankrej (0.9522)
  2. Jersey (0.0337)
  3. Tharparkar (0.0120)

 AUTO-LABELING (Confidence 0.9522 >= 0.9)
 Saved to: /content/unlabelled_images/Kankrej_7_95pct.jpg
 Added to /content/pseudo_labels.csv → Ready for next retraining!


/tmp/ipython-input-3648945213.py:590: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, new_df], ignore_index=True)


('Kankrej', 0.9521840214729309)

In [ ]:
retrain_with_new_data()


🔄 Checking for pseudo-labels...
📥 Found 47 pseudo/human labels — starting retraining...

+++ 🐄 Pseudo Epoch 1/5 +++
📈 Merged Loss: 0.2909

+++ 🐄 Pseudo Epoch 2/5 +++
📈 Merged Loss: 0.3554

+++ 🐄 Pseudo Epoch 3/5 +++
📈 Merged Loss: 0.2782

+++ 🐄 Pseudo Epoch 4/5 +++
📈 Merged Loss: 0.2073

+++ 🐄 Pseudo Epoch 5/5 +++
📈 Merged Loss: 0.2281
✅ Pseudo-training finished – model saved

+++ 🐄 Pseudo Epoch 1/5 +++
📈 Merged Loss: 0.2398

+++ 🐄 Pseudo Epoch 2/5 +++
📈 Merged Loss: 0.1683

+++ 🐄 Pseudo Epoch 3/5 +++
📈 Merged Loss: 0.1603

+++ 🐄 Pseudo Epoch 4/5 +++
📈 Merged Loss: 0.1430

+++ 🐄 Pseudo Epoch 5/5 +++
📈 Merged Loss: 0.1872
✅ Pseudo-training finished – model saved


In [ ]:
!ls -l "/content/Dataset2/Train"

ls: cannot access '/content/Dataset2/Train': No such file or directory


In [ ]:
import os

for breed in os.listdir("/content/Dataset3/Train"):
    count = len(os.listdir(f"/content/Dataset3/Train/{breed}"))
    print(f"train/{breed}: {count} images")

print("\n---")

for breed in os.listdir("/content/Dataset3/Val"):
    count = len(os.listdir(f"/content/Dataset3/Val/{breed}"))
    print(f"val/{breed}: {count} images")

train/Alambadi: 76 images
train/Bargur: 74 images
train/Banni: 86 images
train/Ayrshire: 184 images
train/Amritmahal: 75 images

---
val/Alambadi: 20 images
val/Bargur: 19 images
val/Banni: 22 images
val/Ayrshire: 47 images
val/Amritmahal: 19 images


In [ ]:
!rm "/content/Dataset3.zip"

In [ ]:
"""
EfficientNet-B0 – 85 %+ val, high-confidence, calibrated
- Soft-labels ready for pseudo-labelling
- Temperature scaling inside TTA
"""
import os, json, time, torch, torchvision, numpy as np
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, models
import torchvision.transforms as T
from PIL import Image
from torch.cuda.amp import autocast, GradScaler
from timm.data.auto_augment import rand_augment_transform
from timm.data.mixup import Mixup
from sklearn.utils.class_weight import compute_class_weight

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device}")

# ---------- paths ----------
train_dir   = "/content/New_data_train_val/train"
val_dir     = "/content/New_data_train_val/val"
SAVE_BEST   = "/content/best_model.pth"
SAVE_FINAL  = "/content/breed_model_efficientnet_finetuned.pth"
SAVE_LABELS = "/content/breed_names.json"
TEST_IMAGE  = "/content/New_data_train_val/val/Gir/Gir_168.jpg"

mean, std = [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]

# ---------- augmentation ----------
rand_aug = rand_augment_transform('rand-m7-mstd0.5', {})
train_tf = T.Compose([T.Resize((224, 224)), rand_aug, T.ToTensor(), T.Normalize(mean, std)])

# ---------- test-time 10-crop ----------
val_tf = T.Compose([
    T.Resize(342),
    T.FiveCrop(256),
    T.Lambda(lambda crops: torch.stack([T.ToTensor()(c) for c in crops])),  # 5×3×256×256
    T.Normalize(mean, std)
])

# ---------- datasets ----------
train_ds = datasets.ImageFolder(train_dir, transform=train_tf)
val_ds   = datasets.ImageFolder(val_dir,   transform=val_tf)
breed_names = train_ds.classes

train_loader = DataLoader(train_ds, batch_size=16, shuffle=True, num_workers=2, pin_memory=True)
val_loader   = DataLoader(val_ds,   batch_size=4, shuffle=False, num_workers=2, pin_memory=True)

# ---------- milder mix-up ----------
mixup_fn = Mixup(mixup_alpha=0.1, cutmix_alpha=0.2, prob=1.0,
                 num_classes=len(breed_names), label_smoothing=0.05)

# ---------- model ----------
class Head(nn.Module):
    def __init__(self, in_features, n_classes, drop=0.2):
        super().__init__()
        self.drop = nn.Dropout(drop)
        self.fc   = nn.Linear(in_features, n_classes)
    def forward(self, x):
        return self.fc(self.drop(x))

model = models.efficientnet_b0(weights='IMAGENET1K_V1')
model.classifier = Head(model.classifier[1].in_features, len(breed_names))
model = model.to(device)

# ---------- freeze ----------
for p in model.parameters(): p.requires_grad = False
for p in model.classifier.parameters(): p.requires_grad = True

# ---------- class weights (used only first 10 epochs) ----------
labels = train_ds.targets
weights = torch.tensor(
    compute_class_weight('balanced', classes=np.arange(len(breed_names)), y=labels),
    dtype=torch.float, device=device)

# ---------- optim ----------
optimizer = optim.AdamW(model.classifier.parameters(), lr=3e-3, weight_decay=5e-3)
scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(
                optimizer, T_0=len(train_loader)*5, T_mult=1, eta_min=1e-6)
scaler = GradScaler()

# ---------- helpers ----------
def unfreeze_and_add(stage, lr):
    frozen = []
    for name, module in model.features.named_children():
        if int(name) >= stage:
            for p in module.parameters():
                if not p.requires_grad:
                    p.requires_grad = True
                    frozen.append(p)
    if frozen:
        optimizer.add_param_group({'params': frozen, 'lr': lr})

# ---------- metrics ----------
best_val_loss = 1e9
patience = 15
pat_counter = 0
accum = 2
grad_clip = 1.0
TEMP = 1.5  # temperature for calibration (tuned on val)

# ---------- train ----------
for epoch in range(30):
    print(f"\n----- epoch {epoch+1}/30 -----")
    # ---- unfreeze ----
    if epoch == 3: unfreeze_and_add(6, 3e-4)
    if epoch == 6: unfreeze_and_add(4, 1e-4)
    if epoch == 9: unfreeze_and_add(0, 5e-5)

    # ---- loss choice ----
    criterion = nn.CrossEntropyLoss(weight=weights if epoch < 10 else None, label_smoothing=0.05)

    # ---- train ----
    model.train()
    running = 0.
    for i, (x, y) in enumerate(train_loader):
        x, y = x.to(device, non_blocking=True), y.to(device, non_blocking=True)
        x, y = mixup_fn(x, y)
        with autocast():
            out = model(x)
            loss = criterion(out, y) / accum
        scaler.scale(loss).backward()
        if (i+1) % accum == 0 or i+1 == len(train_loader):
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
            scaler.step(optimizer); scaler.update(); optimizer.zero_grad(set_to_none=True)
        running += loss.item() * accum
    print(f"train loss {running/len(train_loader):.4f}")

    # ---- validate (10-crop) ----
    model.eval()
    val_loss, correct, total = 0., 0, 0
    with torch.no_grad():
        for x, y in val_loader:
            B = y.size(0)
            x = x.view(-1, 3, 256, 256).to(device)  # 5*crops
            y = y.to(device)
            with autocast():
                out = model(x) / TEMP               # temperature scaling
                out = out.view(5, B, -1).mean(0)    # 5-crop average
                val_loss += criterion(out, y).item()
            # horizontal-flip TTA
            out_flip = model(torch.flip(x, dims=[-1])) / TEMP
            out_flip = out_flip.view(5, B, -1).mean(0)
            out = (out + out_flip) / 2
            pred = out.argmax(1)
            correct += (pred == y).sum().item()
            total += B
    val_loss /= len(val_loader)
    val_acc = 100 * correct / total
    print(f"val_loss {val_loss:.4f}  val_acc {val_acc:.2f}%")

    # ---- early stop ----
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), SAVE_BEST)
        pat_counter = 0
    else:
        pat_counter += 1
        if pat_counter >= patience:
            print("early stop"); break
    scheduler.step()

# ---------- save ----------
model.load_state_dict(torch.load(SAVE_BEST))
torch.save(model, SAVE_FINAL)
with open(SAVE_LABELS, "w") as f:
    json.dump(breed_names, f)
print("Saved best model & labels")

# ---------- high-confidence TTA ----------
def tta_predict(image_path, temp=TEMP):
    model.eval()
    img = Image.open(image_path).convert('RGB')
    crops = val_tf(img).to(device)          # 5×3×256×256
    with torch.no_grad():
        with autocast():
            logits = model(crops) / temp    # temperature
            prob = torch.softmax(logits, dim=1).mean(0)
            # flip
            logits_flip = model(torch.flip(crops, dims=[-1])) / temp
            prob_flip = torch.softmax(logits_flip, dim=1).mean(0)
            prob = (prob + prob_flip) / 2
    conf, pred = prob.max(0)
    return breed_names[pred.item()], conf.item()

if os.path.exists(TEST_IMAGE):
    breed, conf = tta_predict(TEST_IMAGE)
    print(f"TTA pred: {breed}  (confidence {conf:.4f})")
else:
    print("Test image not found")

Using cuda

----- epoch 1/30 -----


/tmp/ipython-input-1284013260.py:82: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
/tmp/ipython-input-1284013260.py:121: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


train loss 2.2250


/tmp/ipython-input-1284013260.py:140: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


val_loss 1.8380  val_acc 65.04%

----- epoch 2/30 -----
train loss 1.8831
val_loss 1.7479  val_acc 61.02%

----- epoch 3/30 -----
train loss 1.8314
val_loss 1.7150  val_acc 72.25%

----- epoch 4/30 -----
train loss 1.8613
val_loss 1.5529  val_acc 73.52%

----- epoch 5/30 -----
train loss 1.6910
val_loss 1.4759  val_acc 76.27%

----- epoch 6/30 -----
train loss 1.5096
val_loss 1.4303  val_acc 81.14%

----- epoch 7/30 -----
train loss 1.4599
val_loss 1.4516  val_acc 76.91%

----- epoch 8/30 -----
train loss 1.4490
val_loss 1.4831  val_acc 75.42%

----- epoch 9/30 -----
train loss 1.3494
val_loss 1.4673  val_acc 72.25%

----- epoch 10/30 -----
train loss 1.4370
val_loss 1.4533  val_acc 77.75%

----- epoch 11/30 -----
train loss 1.2593
val_loss 1.3173  val_acc 83.69%

----- epoch 12/30 -----
train loss 1.3305
val_loss 1.3257  val_acc 80.93%

----- epoch 13/30 -----
train loss 1.3175
val_loss 1.3386  val_acc 81.14%

----- epoch 14/30 -----
train loss 1.3199
val_loss 1.3327  val_acc 85.17%



/tmp/ipython-input-1284013260.py:179: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


TTA pred: Gir  (confidence 0.7002)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os, json, time, torch, torchvision, numpy as np, shutil, pandas as pd
from pathlib import Path
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, models
import torchvision.transforms as T
from PIL import Image
from torch.amp import autocast, GradScaler
from timm.data.auto_augment import rand_augment_transform
from timm.data.mixup import Mixup
from sklearn.utils.class_weight import compute_class_weight

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🚀 Using {device}")

# ---------- CONFIG ----------
train_dir      = "/content/drive/MyDrive/New_data_train_val/train"
val_dir        = "/content/drive/MyDrive/New_data_train_val/val"
UNLABELLED_DIR = "/content/unlabelled_images"
PSEUDO_CSV     = "/content/pseudo_labels.csv"
SAVE_BEST      = "/content/best_model.pth"
SAVE_FINAL     = "/content/breed_model_efficientnet_finetuned.pth"
SAVE_LABELS    = "/content/breed_names.json"
TEST_IMAGE     = "/content/drive/MyDrive/New_data_train_val/val/Kankrej/Kankrej_140.jpg"

SOFT_THR       = 0.90
TEMP           = 1.2
RETRAIN_EPOCHS = 5
mean, std      = [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]

# ---------- transforms ----------
rand_aug = rand_augment_transform('rand-m7-mstd0.5', {})
train_tf = T.Compose([T.Resize((224, 224)), rand_aug, T.ToTensor(), T.Normalize(mean, std)])
unlabelled_tf = T.Compose([
    T.Resize(342), T.FiveCrop(256),
    T.Lambda(lambda crops: torch.stack([T.ToTensor()(c) for c in crops])),
    T.Normalize(mean, std)
])
val_tf = T.Compose([
    T.Resize(342), T.FiveCrop(256),
    T.Lambda(lambda crops: torch.stack([T.ToTensor()(c) for c in crops])),
    T.Normalize(mean, std)
])

# ---------- datasets ----------
train_ds = datasets.ImageFolder(train_dir, transform=train_tf)
val_ds   = datasets.ImageFolder(val_dir,   transform=val_tf)
breed_names = train_ds.classes
train_loader = DataLoader(train_ds, batch_size=16, shuffle=True, num_workers=2, pin_memory=True)
val_loader   = DataLoader(val_ds,   batch_size=4, shuffle=False, num_workers=2, pin_memory=True)

# ---------- mixup ----------
mixup_fn = Mixup(mixup_alpha=0.1, cutmix_alpha=0.2, prob=1.0,
                 num_classes=len(breed_names), label_smoothing=0.05)

# ---------- model ----------
class Head(nn.Module):
    def __init__(self, in_features, n_classes, drop=0.2):
        super().__init__()
        self.drop = nn.Dropout(drop)
        self.fc   = nn.Linear(in_features, n_classes)
    def forward(self, x):
        return self.fc(self.drop(x))

model = models.efficientnet_b0(weights='IMAGENET1K_V1')
model.classifier = Head(model.classifier[1].in_features, len(breed_names))
model = model.to(device)

# ---------- freeze ----------
for p in model.parameters(): p.requires_grad = False
for p in model.classifier.parameters(): p.requires_grad = True

# ---------- class weights ----------
labels = train_ds.targets
weights = torch.tensor(
    compute_class_weight('balanced', classes=np.arange(len(breed_names)), y=labels),
    dtype=torch.float, device=device)

# ---------- optim ----------
optimizer = optim.AdamW(model.classifier.parameters(), lr=3e-3, weight_decay=5e-3)
scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(
                optimizer, T_0=len(train_loader)*5, T_mult=1, eta_min=1e-6)
scaler = GradScaler('cuda')

# ---------- helpers ----------
def unfreeze_and_add(stage, lr):
    frozen = []
    for name, module in model.features.named_children():
        if int(name) >= stage:
            for p in module.parameters():
                if not p.requires_grad:
                    p.requires_grad = True
                    frozen.append(p)
    if frozen:
        optimizer.add_param_group({'params': frozen, 'lr': lr})

# ---------- global metrics ----------
best_val_loss = 1e9
patience = 15
pat_counter = 0
accum = 2
grad_clip = 1.0
TEMP = 1.2

# ===================================================================
#  1. MAIN TRAINING
# ===================================================================
def main_training():
    global best_val_loss, pat_counter
    for epoch in range(30):
        print(f"\n----- 🐄 Epoch {epoch+1}/30 -----")
        if epoch == 3: unfreeze_and_add(6, 3e-4)
        if epoch == 6: unfreeze_and_add(4, 1e-4)
        if epoch == 9: unfreeze_and_add(0, 5e-5)

        criterion = nn.CrossEntropyLoss(weight=weights if epoch < 10 else None, label_smoothing=0.05)

        model.train()
        running = 0.
        for i, (x, y) in enumerate(train_loader):
            x, y = x.to(device, non_blocking=True), y.to(device, non_blocking=True)
            x, y = mixup_fn(x, y)
            with autocast('cuda'):
                out = model(x)
                loss = criterion(out, y) / accum
            scaler.scale(loss).backward()
            if (i+1) % accum == 0 or i+1 == len(train_loader):
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
                scaler.step(optimizer); scaler.update(); optimizer.zero_grad(set_to_none=True)
            running += loss.item() * accum
        print(f"📈 Train Loss: {running/len(train_loader):.4f}")

        # ---- validate ----
        model.eval()
        val_loss, correct, total = 0., 0, 0
        with torch.no_grad():
            for x, y in val_loader:
                B = y.size(0)
                x = x.view(-1, 3, 256, 256).to(device)
                y = y.to(device)
                with autocast('cuda'):
                    out = model(x) / TEMP
                    out = out.view(5, B, -1).mean(0)
                    out_flip = model(torch.flip(x, dims=[-1])) / TEMP
                    out_flip = out_flip.view(5, B, -1).mean(0)
                    out = (out + out_flip) / 2
                    val_loss += nn.CrossEntropyLoss()(out, y).item()
                pred = out.argmax(1)
                correct += (pred == y).sum().item()
                total += B
        val_loss /= len(val_loader)
        val_acc = 100 * correct / total
        print(f"🎯 Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), SAVE_BEST)
            pat_counter = 0
            print(f"⭐ New best model saved (val_loss={val_loss:.4f})")
        else:
            pat_counter += 1
            if pat_counter >= patience:
                print("🛑 Early stopping triggered")
                break
        scheduler.step()

    model.load_state_dict(torch.load(SAVE_BEST))
    torch.save(model, SAVE_FINAL)
    with open(SAVE_LABELS, "w") as f:
        json.dump(breed_names, f)
    print("✅ Main training finished – best model saved")

# ===================================================================
#  2. PSEUDO + HUMAN-IN-THE-LOOP
# ===================================================================
def predict_image(image_path, temp=TEMP):
    model.eval()
    img = Image.open(image_path).convert('RGB')
    crops = unlabelled_tf(img).to(device)
    with torch.no_grad():
        with autocast('cuda'):
            logits = model(crops) / temp
            prob = torch.softmax(logits, dim=1).mean(0)
            logits_flip = model(torch.flip(crops, dims=[-1])) / temp
            prob_flip = torch.softmax(logits_flip, dim=1).mean(0)
            prob = (prob + prob_flip) / 2
    top3 = torch.topk(prob, k=3)
    return [(breed_names[i], float(p)) for i, p in zip(top3.indices, top3.values)], prob.cpu()

def worker_choice(image_path, top3):
    print(f"\n📸 Image: {image_path}")
    for idx, (b, p) in enumerate(top3, 1):
        print(f"  {idx}. {b}  ({p*100:.1f}%)")
    while True:
        try:
            choice = input("👉 Choose 1-3 (or 0 to skip): ").strip()
            if choice == "0":
                return None
            choice = int(choice)
            if 1 <= choice <= 3:
                return top3[choice-1][0]
            else:
                print("⚠️ Please enter 0, 1, 2, or 3.")
        except ValueError:
            print("⚠️ Invalid input – please enter a number.")

def create_pseudo_csv():
    unlabelled_path = Path(UNLABELLED_DIR)
    if not unlabelled_path.exists():
        print(f"❌ Unlabelled directory not found: {UNLABELLED_DIR}")
        pd.DataFrame(columns=['file', 'label', 'confidence', 'type']).to_csv(PSEUDO_CSV, index=False)
        return pd.DataFrame()

    # Find image files
    image_files = list(unlabelled_path.rglob("*.[jJ][pP][gG]")) + \
                  list(unlabelled_path.rglob("*.[jJ][pP][eE][gG]")) + \
                  list(unlabelled_path.rglob("*.[pP][nN][gG]"))

    if len(image_files) == 0:
        print(f"⚠️ No images found in {UNLABELLED_DIR}")
        pd.DataFrame(columns=['file', 'label', 'confidence', 'type']).to_csv(PSEUDO_CSV, index=False)
        return pd.DataFrame()

    print(f"🔍 Found {len(image_files)} images to label...")
    records = []

    for img_path in image_files:
        try:
            top3, soft_prob = predict_image(img_path)
            best_breed, best_conf = top3[0]

            if best_conf >= SOFT_THR:
                records.append({
                    'file': str(img_path),
                    'label': best_breed,
                    'confidence': best_conf,
                    'type': 'pseudo'
                })
                print(f"🤖 Auto-labeled: {img_path.name} → {best_breed} ({best_conf:.2f})")
            else:
                chosen = worker_choice(img_path, top3)
                if chosen is not None:
                    records.append({
                        'file': str(img_path),
                        'label': chosen,
                        'confidence': 1.0,
                        'type': 'human'
                    })
                    print(f"👩‍🌾 Human-labeled: {img_path.name} → {chosen}")
                else:
                    print(f"⏭️ Skipped: {img_path.name}")

        except Exception as e:
            print(f"❌ Error processing {img_path}: {e}")
            continue

    df = pd.DataFrame(records)
    df.to_csv(PSEUDO_CSV, index=False)
    print(f"✅ CSV saved → {len(df)} new labels collected")
    return df

# ===================================================================
#  3. SOFT-HARD DATASET + RETRAIN
# ===================================================================
class SoftHardDataset(Dataset):
    def __init__(self, csv_file, root, transform, class_to_idx):
        if not os.path.exists(csv_file):
            self.samples = []
            return
        try:
            self.df = pd.read_csv(csv_file)
        except pd.errors.EmptyDataError:
            self.df = pd.DataFrame()
        self.root = root
        self.transform = transform
        self.class_to_idx = class_to_idx
        self.samples = []
        if len(self.df) > 0:
            for _, row in self.df.iterrows():
                self.samples.append((row['file'], row['label'], row['confidence'], row['type']))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, label, conf, typ = self.samples[idx]
        img = Image.open(path).convert('RGB')
        if self.transform:
            img = self.transform(img)
        if typ == 'pseudo':
            target = torch.zeros(len(self.class_to_idx))
            target[self.class_to_idx[label]] = conf
            return img, target.float()
        else:
            return img, self.class_to_idx[label]

def retrain_with_new_data(extra_epochs=RETRAIN_EPOCHS):
    global accum, grad_clip

    print("\n🔄 Checking for pseudo-labels...")

    # Safety check: if CSV doesn't exist or is empty, skip
    if not os.path.exists(PSEUDO_CSV):
        print("⚠️ Pseudo-labels CSV not found — skipping retraining.")
        return

    try:
        df = pd.read_csv(PSEUDO_CSV)
    except pd.errors.EmptyDataError:
        print("⚠️ Pseudo-labels CSV is empty — skipping retraining.")
        return

    if len(df) == 0:
        print("⚠️ No pseudo-labels collected — skipping retraining.")
        return

    print(f"📥 Found {len(df)} pseudo/human labels — starting retraining...")

    merged_root = "/content/merged_train"
    os.makedirs(merged_root, exist_ok=True)

    # Copy original training data
    os.system(f"cp -r {train_dir}/* {merged_root}/ 2>/dev/null || echo 'Original data copied'")

    # Copy pseudo-labeled images into breed folders
    for _, row in df.iterrows():
        src = Path(row['file'])
        breed = row['label']
        breed_dir = Path(merged_root) / breed
        breed_dir.mkdir(exist_ok=True)
        dst = breed_dir / src.name
        if not dst.exists():
            shutil.copy(src, dst)

    # Create dataset and loader
    merged_ds = SoftHardDataset(PSEUDO_CSV, merged_root, transform=train_tf,
                                class_to_idx=train_ds.class_to_idx)
    if len(merged_ds) == 0:
        print("⚠️ Merged dataset is empty — skipping retraining.")
        return

    merged_loader = DataLoader(merged_ds, batch_size=16, shuffle=True,
                               num_workers=2, pin_memory=True)

    # Add all parameters to optimizer (for fine-tuning)
    optimizer.add_param_group({'params': model.parameters(), 'lr': 1e-4})
    scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(
                    optimizer, T_0=len(merged_loader)*3, T_mult=1, eta_min=1e-6)

    # Custom criterion for soft/hard labels
    def criterion(out, y):
        if y.dtype is torch.float32:
            return -torch.sum(y * torch.log_softmax(out, dim=1), dim=1).mean()
        else:
            return nn.CrossEntropyLoss()(out, y)

    # Retrain loop
    for epoch in range(extra_epochs):
        print(f"\n+++ 🐄 Pseudo Epoch {epoch+1}/{extra_epochs} +++")
        model.train()
        running = 0.
        for x, y in merged_loader:
            x, y = x.to(device, non_blocking=True), y.to(device, non_blocking=True)
            with autocast('cuda'):
                out = model(x)
                loss = criterion(out, y) / accum
            scaler.scale(loss).backward()
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
            scaler.step(optimizer); scaler.update(); optimizer.zero_grad(set_to_none=True)
            running += loss.item() * accum
        print(f"📈 Merged Loss: {running/len(merged_loader):.4f}")

    torch.save(model, "/content/model_after_pseudo.pth")
    print("✅ Pseudo-training finished – model saved")

# ===================================================================
#  4. RUN PIPELINE
# ===================================================================
if __name__ == "__main__":
    print("🏁 Starting full training pipeline...")
    main_training()               # 1. Train on original data
    df = create_pseudo_csv()      # 2. Collect pseudo + human labels
    if len(df) > 0:
        retrain_with_new_data()   # 3. Retrain on merged set (only if we have labels)
    else:
        print("⏭️ Skipping retraining — no new labels collected.")

    print("\n🎉 PIPELINE COMPLETE!")
    print("💾 Final model: /content/breed_model_efficientnet_finetuned.pth")
    print("🏷️  Breed names: /content/breed_names.json")
    if os.path.exists("/content/model_after_pseudo.pth"):
        print("🆙 Upgraded model: /content/model_after_pseudo.pth")

🚀 Using cuda
🏁 Starting full training pipeline...

----- 🐄 Epoch 1/30 -----
📈 Train Loss: 2.1795
🎯 Val Loss: 1.6187 | Val Acc: 65.04%
⭐ New best model saved (val_loss=1.6187)

----- 🐄 Epoch 2/30 -----
📈 Train Loss: 1.8951
🎯 Val Loss: 1.4817 | Val Acc: 71.19%
⭐ New best model saved (val_loss=1.4817)

----- 🐄 Epoch 3/30 -----
📈 Train Loss: 1.8830
🎯 Val Loss: 1.4689 | Val Acc: 72.03%
⭐ New best model saved (val_loss=1.4689)

----- 🐄 Epoch 4/30 -----
📈 Train Loss: 1.7931
🎯 Val Loss: 1.3077 | Val Acc: 72.67%
⭐ New best model saved (val_loss=1.3077)

----- 🐄 Epoch 5/30 -----
📈 Train Loss: 1.6161
🎯 Val Loss: 1.2741 | Val Acc: 69.49%
⭐ New best model saved (val_loss=1.2741)

----- 🐄 Epoch 6/30 -----
📈 Train Loss: 1.5991
🎯 Val Loss: 1.1045 | Val Acc: 77.12%
⭐ New best model saved (val_loss=1.1045)

----- 🐄 Epoch 7/30 -----
📈 Train Loss: 1.6156
🎯 Val Loss: 1.1997 | Val Acc: 74.79%

----- 🐄 Epoch 8/30 -----
📈 Train Loss: 1.4182
🎯 Val Loss: 1.0791 | Val Acc: 78.60%
⭐ New best model saved (val_loss

In [ ]:
# finall improved code by :- deepseek
import os, json, time, torch, torchvision, numpy as np, shutil, pandas as pd
from pathlib import Path
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, models
import torchvision.transforms as T
from PIL import Image
from torch.amp import autocast, GradScaler
from timm.data.auto_augment import rand_augment_transform
from timm.data.mixup import Mixup
from sklearn.utils.class_weight import compute_class_weight
import cv2
from datetime import datetime

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🚀 Using {device}")

# ---------- CONFIG ----------
train_dir      = "/content/drive/MyDrive/New_data_train_val/train"
val_dir        = "/content/drive/MyDrive/New_data_train_val/val"
UNLABELLED_DIR = "/content/unlabelled_images"
PROCESSED_DIR = "/content/processed_images"  # To store images after processing
PSEUDO_CSV     = "/content/pseudo_labels.csv"
SAVE_BEST      = "/content/best_model.pth"
SAVE_FINAL     = "/content/breed_model_efficientnet_finetuned.pth"
SAVE_LABELS    = "/content/breed_names.json"
TEST_IMAGE     = "/content/drive/MyDrive/New_data_train_val/val/Sahiwal/Sahiwal_155.jpg"

SOFT_THR       = 0.90  # Confidence threshold for auto-labeling
TEMP           = 1.2
RETRAIN_EPOCHS = 5
mean, std      = [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]

# Create directories if they don't exist
os.makedirs(UNLABELLED_DIR, exist_ok=True)
os.makedirs(PROCESSED_DIR, exist_ok=True)

# ---------- transforms ----------
rand_aug = rand_augment_transform('rand-m7-mstd0.5', {})
train_tf = T.Compose([T.Resize((224, 224)), rand_aug, T.ToTensor(), T.Normalize(mean, std)])
unlabelled_tf = T.Compose([
    T.Resize(342), T.FiveCrop(256),
    T.Lambda(lambda crops: torch.stack([T.ToTensor()(c) for c in crops])),
    T.Normalize(mean, std)
])
val_tf = T.Compose([
    T.Resize(342), T.FiveCrop(256),
    T.Lambda(lambda crops: torch.stack([T.ToTensor()(c) for c in crops])),
    T.Normalize(mean, std)
])

# ---------- datasets ----------
train_ds = datasets.ImageFolder(train_dir, transform=train_tf)
val_ds   = datasets.ImageFolder(val_dir,   transform=val_tf)
breed_names = train_ds.classes
train_loader = DataLoader(train_ds, batch_size=16, shuffle=True, num_workers=2, pin_memory=True)
val_loader   = DataLoader(val_ds,   batch_size=4, shuffle=False, num_workers=2, pin_memory=True)

# ---------- mixup ----------
mixup_fn = Mixup(mixup_alpha=0.1, cutmix_alpha=0.2, prob=1.0,
                 num_classes=len(breed_names), label_smoothing=0.05)

# ---------- model ----------
class Head(nn.Module):
    def __init__(self, in_features, n_classes, drop=0.2):
        super().__init__()
        self.drop = nn.Dropout(drop)
        self.fc   = nn.Linear(in_features, n_classes)
    def forward(self, x):
        return self.fc(self.drop(x))

model = models.efficientnet_b0(weights='IMAGENET1K_V1')
model.classifier = Head(model.classifier[1].in_features, len(breed_names))
model = model.to(device)

# ---------- freeze ----------
for p in model.parameters(): p.requires_grad = False
for p in model.classifier.parameters(): p.requires_grad = True

# ---------- class weights ----------
labels = train_ds.targets
weights = torch.tensor(
    compute_class_weight('balanced', classes=np.arange(len(breed_names)), y=labels),
    dtype=torch.float, device=device)

# ---------- optim ----------
optimizer = optim.AdamW(model.classifier.parameters(), lr=3e-3, weight_decay=5e-3)
scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(
                optimizer, T_0=len(train_loader)*5, T_mult=1, eta_min=1e-6)
scaler = GradScaler('cuda')

# ---------- helpers ----------
def unfreeze_and_add(stage, lr):
    frozen = []
    for name, module in model.features.named_children():
        if int(name) >= stage:
            for p in module.parameters():
                if not p.requires_grad:
                    p.requires_grad = True
                    frozen.append(p)
    if frozen:
        optimizer.add_param_group({'params': frozen, 'lr': lr})

# ---------- global metrics ----------
best_val_loss = 1e9
patience = 15
pat_counter = 0
accum = 2
grad_clip = 1.0
TEMP = 1.2

# ===================================================================
#  CAPTURE/STORE IMAGES FROM FLWs
# ===================================================================
def capture_and_store_image(image_path=None, image_array=None, camera_index=0):
    """
    Capture an image from camera or save provided image to unlabelled folder
    """
    if image_path and os.path.exists(image_path):
        # Copy provided image to unlabelled folder
        filename = os.path.basename(image_path)
        dest_path = os.path.join(UNLABELLED_DIR, f"{int(time.time())}_{filename}")
        shutil.copy(image_path, dest_path)
        print(f"📁 Image saved to {dest_path}")
        return dest_path
    elif image_array is not None:
        # Save image array to unlabelled folder
        filename = f"captured_{int(time.time())}.jpg"
        dest_path = os.path.join(UNLABELLED_DIR, filename)
        cv2.imwrite(dest_path, image_array)
        print(f"📁 Image saved to {dest_path}")
        return dest_path
    else:
        # Capture from camera
        cap = cv2.VideoCapture(camera_index)
        ret, frame = cap.read()
        cap.release()

        if ret:
            filename = f"captured_{int(time.time())}.jpg"
            dest_path = os.path.join(UNLABELLED_DIR, filename)
            cv2.imwrite(dest_path, frame)
            print(f"📁 Image captured and saved to {dest_path}")
            return dest_path
        else:
            print("❌ Failed to capture image from camera")
            return None

# ===================================================================
#  1. MAIN TRAINING
# ===================================================================
def main_training():
    global best_val_loss, pat_counter
    for epoch in range(30):
        print(f"\n----- 🐄 Epoch {epoch+1}/30 -----")
        if epoch == 3: unfreeze_and_add(6, 3e-4)
        if epoch == 6: unfreeze_and_add(4, 1e-4)
        if epoch == 9: unfreeze_and_add(0, 5e-5)

        criterion = nn.CrossEntropyLoss(weight=weights if epoch < 10 else None, label_smoothing=0.05)

        model.train()
        running = 0.
        for i, (x, y) in enumerate(train_loader):
            x, y = x.to(device, non_blocking=True), y.to(device, non_blocking=True)
            x, y = mixup_fn(x, y)
            with autocast('cuda'):
                out = model(x)
                loss = criterion(out, y) / accum
            scaler.scale(loss).backward()
            if (i+1) % accum == 0 or i+1 == len(train_loader):
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
                scaler.step(optimizer); scaler.update(); optimizer.zero_grad(set_to_none=True)
            running += loss.item() * accum
        print(f"📈 Train Loss: {running/len(train_loader):.4f}")

        # ---- validate ----
        model.eval()
        val_loss, correct, total = 0., 0, 0
        with torch.no_grad():
            for x, y in val_loader:
                B = y.size(0)
                x = x.view(-1, 3, 256, 256).to(device)
                y = y.to(device)
                with autocast('cuda'):
                    out = model(x) / TEMP
                    out = out.view(5, B, -1).mean(0)
                    out_flip = model(torch.flip(x, dims=[-1])) / TEMP
                    out_flip = out_flip.view(5, B, -1).mean(0)
                    out = (out + out_flip) / 2
                    val_loss += nn.CrossEntropyLoss()(out, y).item()
                pred = out.argmax(1)
                correct += (pred == y).sum().item()
                total += B
        val_loss /= len(val_loader)
        val_acc = 100 * correct / total
        print(f"🎯 Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), SAVE_BEST)
            pat_counter = 0
            print(f"⭐ New best model saved (val_loss={val_loss:.4f})")
        else:
            pat_counter += 1
            if pat_counter >= patience:
                print("🛑 Early stopping triggered")
                break
        scheduler.step()

    model.load_state_dict(torch.load(SAVE_BEST))
    torch.save(model, SAVE_FINAL)
    with open(SAVE_LABELS, "w") as f:
        json.dump(breed_names, f)
    print("✅ Main training finished – best model saved")

# ===================================================================
#  2. PSEUDO + HUMAN-IN-THE-LOOP
# ===================================================================
def predict_image(image_path, temp=TEMP):
    model.eval()
    img = Image.open(image_path).convert('RGB')
    crops = unlabelled_tf(img).to(device)
    with torch.no_grad():
        with autocast('cuda'):
            logits = model(crops) / temp
            prob = torch.softmax(logits, dim=1).mean(0)
            logits_flip = model(torch.flip(crops, dims=[-1])) / temp
            prob_flip = torch.softmax(logits_flip, dim=1).mean(0)
            prob = (prob + prob_flip) / 2
    top3 = torch.topk(prob, k=3)
    return [(breed_names[i], float(p)) for i, p in zip(top3.indices, top3.values)], prob.cpu()

def worker_choice(image_path, top3):
    print(f"\n📸 Image: {os.path.basename(image_path)}")
    for idx, (b, p) in enumerate(top3, 1):
        print(f"  {idx}. {b}  ({p*100:.1f}%)")
    print("  0. Skip this image")
    print("  4. None of the above")

    while True:
        try:
            choice = input("👉 Choose 0-4: ").strip()
            if choice == "0":
                return "skip"
            elif choice == "4":
                return "none"
            choice = int(choice)
            if 1 <= choice <= 3:
                return top3[choice-1][0]
            else:
                print("⚠️ Please enter 0, 1, 2, 3, or 4.")
        except ValueError:
            print("⚠️ Invalid input – please enter a number.")

def create_pseudo_csv():
    unlabelled_path = Path(UNLABELLED_DIR)
    if not unlabelled_path.exists():
        print(f"❌ Unlabelled directory not found: {UNLABELLED_DIR}")
        pd.DataFrame(columns=['file', 'label', 'confidence', 'type', 'date']).to_csv(PSEUDO_CSV, index=False)
        return pd.DataFrame()

    # Find image files
    image_files = list(unlabelled_path.rglob("*.[jJ][pP][gG]")) + \
                  list(unlabelled_path.rglob("*.[jJ][pP][eE][gG]")) + \
                  list(unlabelled_path.rglob("*.[pP][nN][gG]"))

    if len(image_files) == 0:
        print(f"⚠️ No images found in {UNLABELLED_DIR}")
        pd.DataFrame(columns=['file', 'label', 'confidence', 'type', 'date']).to_csv(PSEUDO_CSV, index=False)
        return pd.DataFrame()

    print(f"🔍 Found {len(image_files)} images to label...")
    records = []

    # Load existing labels (if any) to avoid reprocessing
    existing_files = set()
    if os.path.exists(PSEUDO_CSV):
        try:
            existing_df = pd.read_csv(PSEUDO_CSV)
            existing_files = set(existing_df['file'].tolist())
        except:
            pass

    for img_path in image_files:
        # Skip if already labeled
        if str(img_path) in existing_files:
            print(f"⏭️ Already labeled: {img_path.name}")
            continue

        try:
            top3, soft_prob = predict_image(img_path)
            best_breed, best_conf = top3[0]

            if best_conf >= SOFT_THR:
                # Auto-label with high confidence
                records.append({
                    'file': str(img_path),
                    'label': best_breed,
                    'confidence': best_conf,
                    'type': 'pseudo',
                    'date': datetime.now().isoformat()
                })
                print(f"🤖 Auto-labeled: {img_path.name} → {best_breed} ({best_conf:.2f})")

                # Move to processed folder
                processed_path = os.path.join(PROCESSED_DIR, img_path.name)
                shutil.move(str(img_path), processed_path)

            else:
                # Get human input for low confidence predictions
                chosen = worker_choice(img_path, top3)

                if chosen == "skip":
                    print(f"⏭️ Skipped: {img_path.name}")
                    # Move to processed folder without labeling
                    processed_path = os.path.join(PROCESSED_DIR, img_path.name)
                    shutil.move(str(img_path), processed_path)

                elif chosen == "none":
                    print(f"🏷️ Manual label needed for: {img_path.name}")
                    manual_label = input("👉 Enter the breed name manually: ").strip()
                    if manual_label and manual_label in breed_names:
                        records.append({
                            'file': str(img_path),
                            'label': manual_label,
                            'confidence': 1.0,
                            'type': 'human',
                            'date': datetime.now().isoformat()
                        })
                        print(f"👩‍🌾 Human-labeled: {img_path.name} → {manual_label}")

                        # Move to processed folder
                        processed_path = os.path.join(PROCESSED_DIR, img_path.name)
                        shutil.move(str(img_path), processed_path)
                    else:
                        print(f"❌ Invalid breed name. Skipping: {img_path.name}")
                        # Move to processed folder without labeling
                        processed_path = os.path.join(PROCESSED_DIR, img_path.name)
                        shutil.move(str(img_path), processed_path)

                else:
                    records.append({
                        'file': str(img_path),
                        'label': chosen,
                        'confidence': 1.0,
                        'type': 'human',
                        'date': datetime.now().isoformat()
                    })
                    print(f"👩‍🌾 Human-labeled: {img_path.name} → {chosen}")

                    # Move to processed folder
                    processed_path = os.path.join(PROCESSED_DIR, img_path.name)
                    shutil.move(str(img_path), processed_path)

        except Exception as e:
            print(f"❌ Error processing {img_path}: {e}")
            continue

    # Merge with existing CSV
    if os.path.exists(PSEUDO_CSV) and len(records) > 0:
        try:
            existing_df = pd.read_csv(PSEUDO_CSV)
            new_df = pd.DataFrame(records)
            df = pd.concat([existing_df, new_df], ignore_index=True)
        except:
            df = pd.DataFrame(records)
    else:
        df = pd.DataFrame(records)

    df.to_csv(PSEUDO_CSV, index=False)
    print(f"✅ CSV saved → {len(df)} total labels collected")
    return df

# ===================================================================
#  3. SOFT-HARD DATASET + RETRAIN
# ===================================================================
class SoftHardDataset(Dataset):
    def __init__(self, csv_file, root, transform, class_to_idx):
        if not os.path.exists(csv_file):
            self.samples = []
            return
        try:
            self.df = pd.read_csv(csv_file)
        except pd.errors.EmptyDataError:
            self.df = pd.DataFrame()
        self.root = root
        self.transform = transform
        self.class_to_idx = class_to_idx
        self.samples = []
        if len(self.df) > 0:
            for _, row in self.df.iterrows():
                # Check if file exists (it might have been moved to processed folder)
                file_path = row['file']
                if not os.path.exists(file_path):
                    # Try to find it in the processed folder
                    processed_path = os.path.join(PROCESSED_DIR, os.path.basename(file_path))
                    if os.path.exists(processed_path):
                        file_path = processed_path

                self.samples.append((file_path, row['label'], row['confidence'], row['type']))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, label, conf, typ = self.samples[idx]
        img = Image.open(path).convert('RGB')
        if self.transform:
            img = self.transform(img)
        if typ == 'pseudo':
            target = torch.zeros(len(self.class_to_idx))
            target[self.class_to_idx[label]] = conf
            return img, target.float()
        else:
            return img, self.class_to_idx[label]

def retrain_with_new_data(extra_epochs=RETRAIN_EPOCHS):
    global accum, grad_clip

    print("\n🔄 Checking for pseudo-labels...")

    # Safety check: if CSV doesn't exist or is empty, skip
    if not os.path.exists(PSEUDO_CSV):
        print("⚠️ Pseudo-labels CSV not found — skipping retraining.")
        return

    try:
        df = pd.read_csv(PSEUDO_CSV)
    except pd.errors.EmptyDataError:
        print("⚠️ Pseudo-labels CSV is empty — skipping retraining.")
        return

    if len(df) == 0:
        print("⚠️ No pseudo-labels collected — skipping retraining.")
        return

    print(f"📥 Found {len(df)} pseudo/human labels — starting retraining...")

    merged_root = "/content/merged_train"
    os.makedirs(merged_root, exist_ok=True)

    # Copy original training data
    os.system(f"cp -r {train_dir}/* {merged_root}/ 2>/dev/null || echo 'Original data copied'")

    # Copy pseudo-labeled images into breed folders
    for _, row in df.iterrows():
        # Check if file exists in original location or processed folder
        src_path = row['file']
        if not os.path.exists(src_path):
            processed_path = os.path.join(PROCESSED_DIR, os.path.basename(src_path))
            if os.path.exists(processed_path):
                src_path = processed_path
            else:
                print(f"⚠️ Could not find image: {src_path}")
                continue

        breed = row['label']
        breed_dir = Path(merged_root) / breed
        breed_dir.mkdir(exist_ok=True)
        dst = breed_dir / os.path.basename(src_path)
        if not dst.exists():
            shutil.copy(src_path, dst)

    # Create dataset and loader
    merged_ds = SoftHardDataset(PSEUDO_CSV, merged_root, transform=train_tf,
                                class_to_idx=train_ds.class_to_idx)
    if len(merged_ds) == 0:
        print("⚠️ Merged dataset is empty — skipping retraining.")
        return

    merged_loader = DataLoader(merged_ds, batch_size=16, shuffle=True,
                               num_workers=2, pin_memory=True)

    # Add all parameters to optimizer (for fine-tuning)
    optimizer.add_param_group({'params': model.parameters(), 'lr': 1e-4})
    scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(
                    optimizer, T_0=len(merged_loader)*3, T_mult=1, eta_min=1e-6)

    # Custom criterion for soft/hard labels
    def criterion(out, y):
        if y.dtype is torch.float32:
            return -torch.sum(y * torch.log_softmax(out, dim=1), dim=1).mean()
        else:
            return nn.CrossEntropyLoss()(out, y)

    # Retrain loop
    for epoch in range(extra_epochs):
        print(f"\n+++ 🐄 Pseudo Epoch {epoch+1}/{extra_epochs} +++")
        model.train()
        running = 0.
        for x, y in merged_loader:
            x, y = x.to(device, non_blocking=True), y.to(device, non_blocking=True)
            with autocast('cuda'):
                out = model(x)
                loss = criterion(out, y) / accum
            scaler.scale(loss).backward()
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
            scaler.step(optimizer); scaler.update(); optimizer.zero_grad(set_to_none=True)
            running += loss.item() * accum
        print(f"📈 Merged Loss: {running/len(merged_loader):.4f}")

    torch.save(model, "/content/model_after_pseudo.pth")
    print("✅ Pseudo-training finished – model saved")

# ===================================================================
#  4. HELPER: LABEL NEW IMAGES ONLY (NO RETRAIN)
# ===================================================================
def label_new_images_only():
    """Run ONLY the labeling step — useful when adding new test photos"""
    print("📸 Starting NEW IMAGE LABELING only...")

    # Load model if not already loaded
    global model, breed_names
    if 'model' not in globals() or model is None:
        print("Loading model...")
        model = torch.load(SAVE_FINAL)
        model = model.to(device)
        model.eval()
        with open(SAVE_LABELS, "r") as f:
            breed_names = json.load(f)
        print("Model loaded.")

    df = create_pseudo_csv()  # This will label only NEW images
    print(f"✅ Done labeling. Total labeled images: {len(df) if not df.empty else 0}")
    return df

# ===================================================================
#  5. EVALUATE SINGLE IMAGE
# ===================================================================
def evaluate_single_image(image_path=None, image_array=None, camera_index=0):
    """
    Evaluate a single image with the model
    Returns: prediction, confidence, top choices
    """
    # Capture/store image if needed
    if image_path or image_array is not None or camera_index >= 0:
        stored_path = capture_and_store_image(image_path, image_array, camera_index)
        if stored_path:
            image_path = stored_path

    if not image_path or not os.path.exists(image_path):
        print("❌ No valid image provided")
        return None, None, None

    # Make prediction
    top3, probs = predict_image(image_path)
    prediction, confidence = top3[0]

    print(f"\n🔍 Prediction for {os.path.basename(image_path)}:")
    print(f"   Breed: {prediction} (Confidence: {confidence:.2%})")
    print("\n📊 Top choices:")
    for i, (breed, conf) in enumerate(top3, 1):
        print(f"   {i}. {breed}: {conf:.2%}")

    return prediction, confidence, top3

# ===================================================================
#  6. RUN PIPELINE
# ===================================================================
if __name__ == "__main__":
    print("🏁 Starting full training pipeline...")
    main_training()               # 1. Train on original data
    df = create_pseudo_csv()      # 2. Collect pseudo + human labels
    if len(df) > 0:
        retrain_with_new_data()   # 3. Retrain on merged set (only if we have labels)
    else:
        print("⏭️ Skipping retraining — no new labels collected.")

    print("\n🎉 PIPELINE COMPLETE!")
    print("💾 Final model: /content/breed_model_efficientnet_finetuned.pth")
    print("🏷️  Breed names: /content/breed_names.json")
    if os.path.exists("/content/model_after_pseudo.pth"):
        print("🆙 Upgraded model: /content/model_after_pseudo.pth")

    print("\n💡 TIP: To label NEW images later, run:")
    print("   df = label_new_images_only()")
    print("   if len(df) > 0:")
    print("       retrain_with_new_data()")

    print("\n📸 To evaluate a single image, run:")
    print("   evaluate_single_image(image_path='path/to/image.jpg')")
    print("   evaluate_single_image(camera_index=0)  # Capture from camera")

🚀 Using cuda
🏁 Starting full training pipeline...

----- 🐄 Epoch 1/30 -----
📈 Train Loss: 2.1953
🎯 Val Loss: 1.6545 | Val Acc: 64.19%
⭐ New best model saved (val_loss=1.6545)

----- 🐄 Epoch 2/30 -----
📈 Train Loss: 1.9858
🎯 Val Loss: 1.5423 | Val Acc: 68.22%
⭐ New best model saved (val_loss=1.5423)

----- 🐄 Epoch 3/30 -----
📈 Train Loss: 1.9345
🎯 Val Loss: 1.5151 | Val Acc: 72.03%
⭐ New best model saved (val_loss=1.5151)

----- 🐄 Epoch 4/30 -----
📈 Train Loss: 1.8796
🎯 Val Loss: 1.2695 | Val Acc: 77.33%
⭐ New best model saved (val_loss=1.2695)

----- 🐄 Epoch 5/30 -----
📈 Train Loss: 1.6944
🎯 Val Loss: 1.2483 | Val Acc: 76.27%
⭐ New best model saved (val_loss=1.2483)

----- 🐄 Epoch 6/30 -----
📈 Train Loss: 1.6316
🎯 Val Loss: 1.1942 | Val Acc: 74.58%
⭐ New best model saved (val_loss=1.1942)

----- 🐄 Epoch 7/30 -----
📈 Train Loss: 1.5284
🎯 Val Loss: 1.1339 | Val Acc: 76.91%
⭐ New best model saved (val_loss=1.1339)

----- 🐄 Epoch 8/30 -----
📈 Train Loss: 1.3936
🎯 Val Loss: 1.0406 | Val Acc

KeyboardInterrupt: 